In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt

In [2]:
def biohashing(img_vec, M, m, k, thresh):
    # Step 1: Generate pseudo-random vectors {ri} based on user's seed k
    np.random.seed(k)
    r = np.random.rand(m, M)
    
    # Step 2: Apply Gram-Schmidt process to transform r into an orthonormal set of matrices
    r_orthogonal = np.array([r[0]])
    for i in range(1, m):
        ri = r[i]
        for j in range(i):
            ri -= np.dot(r[i], r_orthogonal[j]) * r_orthogonal[j]
        ri /= np.linalg.norm(ri)
        r_orthogonal = np.vstack([r_orthogonal, ri])
    
    # Step 3: Compute ⟨img_vec, r⊥i⟩ for each r⊥i
    inner_products = np.dot(img_vec, r_orthogonal.T)
    
    # Step 4: Compute m bits BioHash {bi}
    bi = []
    for i in range(m):
        if inner_products[i] <= thresh:
            bi.append(0)
        else:
            bi.append(1)
    
    return bi

In [30]:
image_path = os.path.join(r'datasets\augmented_jabcode\user_1\jabcode_user_1_augmented_0.png')
image = cv2.imread(image_path)
image = cv2.resize(image, (256, 256))

image_vector = image / 255.0  # Normalize pixel values
img_vec = image_vector.ravel().copy()

m = (1 * 1 * 256)
k = 202211050
thresh = 0.4

M = len(img_vec)

protected_template = biohashing(img_vec, M, m, k, thresh)

KeyboardInterrupt: 

In [24]:
str(protected_template)

'[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1]'

In [37]:
# Directory containing user folders
data_dir = r'datasets\augmented_jabcode'

# Biohashing parameters
m = (1 * 1 * 256)
thresh = 0.4

In [32]:
# Initialize your encoder-decoder model
def create_encoder_decoder_model(input_shape):
    model = models.Sequential()
    
    # Encoder
    model.add(layers.Conv2D(16, (3, 3), strides=(2, 2), padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv2D(32, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU(name='encoder_op'))
    
    
    # Decoder
    model.add(layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    model.add(layers.Conv2DTranspose(3, (3, 3), strides=(2, 2), padding='same', activation='sigmoid'))
    
    return model

# Auto-encoder loss functions
def autoencoder_loss_bce(y_true, y_pred):
    return tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)

def autoencoder_loss_l2(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

# Triplet loss function
def triplet_loss(alpha, beta=1.0):
    def loss(y_true, y_pred):
        anchor, positive, negative = y_pred
        pos_distance = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
        neg_distance = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
        loss_value = tf.maximum(0.0, pos_distance - neg_distance + beta)
        return alpha * tf.reduce_mean(loss_value)
    return loss


# Initialize your encoder-decoder model for this user
input_shape = (256, 256, 3)
encoder_decoder_model = create_encoder_decoder_model(input_shape)

encoder_decoder_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_83 (Conv2D)          (None, 128, 128, 16)      448       
                                                                 
 batch_normalization_134 (Ba  (None, 128, 128, 16)     64        
 tchNormalization)                                               
                                                                 
 re_lu_122 (ReLU)            (None, 128, 128, 16)      0         
                                                                 
 conv2d_84 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 batch_normalization_135 (Ba  (None, 64, 64, 32)       128       
 tchNormalization)                                               
                                                                 
 re_lu_123 (ReLU)            (None, 64, 64, 32)      

In [27]:
encoder_last_layer = 'encoder_op'
index = None
for idx, layer in enumerate(encoder_decoder_model.layers):
    if layer.name == encoder_last_layer:
        index = idx
        break
index

23

In [38]:
# Main loop to process user folders
for user_id in range(1, 11):
    user_folder = os.path.join(data_dir, f'user_{user_id}')

    set_biohash = set()
    count = 0

    # Initialize the user_id_input with the current user_id
    user_id_input = np.array([user_id])
    
    
    # Iterate through images in the user folder
    for file_name in os.listdir(user_folder):
        image_path = os.path.join(user_folder, file_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (256, 256))
        # Biohashing
        image = image / 255.0  # Normalize pixel values
        img_vec = image.ravel().copy()

        #m = 10# Replace with the desired length of the protected template
        k = 202211050 # Replace with the user's seed
        thresh = 0.4  # Replace with the preset threshold

        M = len(img_vec)
        protected_template = biohashing(img_vec, M, m, k, thresh)
        
        # Update k for the next iteration (for different biohashes)
        k += 100

        # Save the biohash for this image
        binary_number = protected_template
        set_biohash.add(str(binary_number))
        
        # Add the biohash to the encoded layer
        from keras import backend as K

        # with a Sequential model
        get_layer_op = K.function([encoder_decoder_model.layers[0].input], [encoder_decoder_model.layers[index].output])
        # print(get_layer_op)
        encoder_inp = tf.expand_dims(image, axis=0)
        encoded = get_layer_op([encoder_inp])[0]
        
        print(f'encoded shape: {encoded.shape}')
        print(np.array(binary_number).shape)
        add_val = np.add(encoded.ravel(), binary_number)
        print(f'add_val: {add_val}')
        print()
        # Compile and train the model for this user
    alpha = 0.5
    triplet_margin = 1.0
    encoder_decoder_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss={
            'input': autoencoder_loss_bce,
            'user_id': triplet_loss(alpha, beta=triplet_margin)
        }
    )

    # Train the model
    epochs = 10
    encoder_decoder_model.fit(
        {'input': train_data, 'user_id': train_data.labels},
        validation_data=({'input': validation_data, 'user_id': validation_data.labels}),
        epochs=epochs
    )

encoded shape: (1, 1, 1, 256)
(256,)
add_val: [1.00866600e+00 8.74114502e-03 0.00000000e+00 1.00000000e+00
 1.01351542e+00 0.00000000e+00 7.39133800e-04 0.00000000e+00
 6.69054221e-03 0.00000000e+00 3.59421060e-03 1.72184668e-02
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.42266129e-03 0.00000000e+00 6.75499812e-03
 7.96667393e-03 0.00000000e+00 1.47202294e-02 9.43606999e-03
 0.00000000e+00 8.79168976e-03 1.00040632e+00 1.00000000e+00
 1.00367024e+00 1.00263871e+00 0.00000000e+00 1.00016445e+00
 0.00000000e+00 7.95983523e-03 0.00000000e+00 0.00000000e+00
 3.40408483e-03 0.00000000e+00 0.00000000e+00 1.00000000e+00
 1.00000000e+00 0.00000000e+00 1.00000000e+00 1.00770725e+00
 1.78311940e-03 1.26513653e-02 4.30294964e-03 2.10946612e-03
 0.00000000e+00 0.00000000e+00 1.00000000e+00 1.00000000e+00
 0.00000000e+00 8.18008278e-03 0.00000000e+00 1.00255982e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 3.28171090e-03
 0.00000000e+00 0.00000000e+00 6.502827

encoded shape: (1, 1, 1, 256)
(256,)
add_val: [1.00139752e+00 4.35354514e-03 8.45608953e-03 0.00000000e+00
 9.83653031e-03 0.00000000e+00 1.03318430e-02 0.00000000e+00
 1.31579128e-03 0.00000000e+00 6.26360765e-04 1.85139738e-02
 1.00226618e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.88972026e-03 1.21735595e-02
 1.32649001e-02 0.00000000e+00 1.63796451e-02 1.34881288e-02
 1.00000000e+00 6.78717531e-03 0.00000000e+00 4.46507707e-03
 4.47255699e-03 4.15436400e-04 1.00000000e+00 1.88511942e-04
 1.00000000e+00 9.52393562e-03 0.00000000e+00 1.00000000e+00
 1.00195528e+00 0.00000000e+00 1.00000000e+00 5.62433677e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 7.23752240e-03
 2.56131613e-03 8.37158132e-03 1.00436183e+00 0.00000000e+00
 0.00000000e+00 3.05842445e-03 0.00000000e+00 1.00000000e+00
 1.00000000e+00 1.52317751e-02 0.00000000e+00 1.91832834e-03
 0.00000000e+00 1.82160339e-03 0.00000000e+00 1.33611250e-03
 1.00000000e+00 1.00061608e+00 9.031673

encoded shape: (1, 1, 1, 256)
(256,)
add_val: [1.00664862e+00 3.66082136e-03 6.67148829e-03 0.00000000e+00
 8.53221118e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 4.44100797e-03 0.00000000e+00 0.00000000e+00 1.09970486e-02
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.62667334e-03
 1.37335798e-02 1.00128349e+00 1.99414063e-02 9.53522883e-03
 1.00000000e+00 1.01241084e+00 1.00000000e+00 3.24714836e-03
 3.28120869e-03 1.00790603e+00 0.00000000e+00 3.47213913e-03
 0.00000000e+00 1.65511798e-02 0.00000000e+00 0.00000000e+00
 1.00000000e+00 1.00000000e+00 1.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 7.30249146e-03
 6.13746187e-03 1.09576257e-02 7.18781957e-05 1.00000000e+00
 0.00000000e+00 3.57241696e-03 0.00000000e+00 1.00000000e+00
 0.00000000e+00 6.93515176e-03 0.00000000e+00 3.24435509e-03
 0.00000000e+00 4.36172122e-04 0.00000000e+00 6.31388393e-04
 1.00074421e+00 2.80928016e-05 1.018660

encoded shape: (1, 1, 1, 256)
(256,)
add_val: [1.00565800e+00 8.83433968e-03 0.00000000e+00 1.00000000e+00
 9.10122041e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 7.57243857e-03 1.48649712e-03 1.00909193e+00 1.37231825e-02
 1.77574041e-03 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 1.00000000e+00 1.24853896e-02
 5.75365219e-03 1.00000000e+00 1.03356531e-02 1.27124423e-02
 1.00000000e+00 4.95809084e-03 1.00005123e+00 5.77648025e-05
 6.33058371e-03 0.00000000e+00 0.00000000e+00 5.07591642e-04
 0.00000000e+00 1.28472261e-02 0.00000000e+00 0.00000000e+00
 3.93047137e-03 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00972812e+00
 1.00236844e-04 1.45229716e-02 6.82811486e-03 5.71998768e-03
 0.00000000e+00 0.00000000e+00 1.00000000e+00 1.00457939e+00
 1.00000000e+00 5.80665981e-03 0.00000000e+00 1.00000000e+00
 1.00000000e+00 1.78533758e-03 0.00000000e+00 1.35675585e-03
 0.00000000e+00 1.00000000e+00 7.754558

NameError: name 'train_data' is not defined